# Projeto 1 - Ciência dos Dados

Nome: Fernando Bichuette Assumpção

Nome: Felipe Lacombe 

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
import emoji
import functools
import operator
from time import sleep
from random import shuffle

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Fernando\Documents\Insper\3° Semestre\Ciencia dos Dados\C_Dados_Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
#importando database
dados=pd.read_excel("bolsa.xlsx",'Treinamento')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

No projeto 1 de Ciência de dados será feito um clasificador que vai analizar a palavra bolsa em todos os tweets e classificar se é relevante quando está relacionada a **bolsa de valores** e irrelevante quando é empregada em outro em outro sentido ex: bolsa familia, bolsa utilitaria e etc.


## Limpando database

In [4]:
dados.B1.value_counts()

1    153
0    147
Name: B1, dtype: int64

In [43]:
#fazendo a limpeza do arquivo
"""
def limpar_caracteres(tabela, titulo):
    i = tabela[titulo]
    i = i.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t','@','?','%',]
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            i = i.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    for e in itens:
        i = i.str.replace(e,'')
    for e in i2:
        i = i.str.replace(e,' {} '.format(e))

    i = i.str.replace('é','e')
    i = i.str.replace('ê','e')
    i = i.str.replace('á','a')
    i = i.str.replace('ã','a')
    i = i.str.replace('ô','o')
    i = i.str.replace('ó','o')
    i = i.str.replace('ú','u')
    i = i.str.replace('ç','c')
    i = i.str.replace('í','i')
    i = i.str.replace('@','')
    i = i.str.replace('?','')
    i = i.str.replace(',','.')
        
    tabela[titulo] = i

    return tabela


titulo="Treinamento"

tabela=limpar_caracteres(dados,titulo)

def separador(tabela, titulo):
    i = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['Tabela splitted'] = []

    for lista in i:
        for e in ['#','' , ' ', 'rt']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'Tabela splitted': lista}, ignore_index=True)
                
    return nova
nova=separador(dados,titulo)

def tabela_palavras(tabela):
    a = tabela['Tabela splitted']
    prob = pd.DataFrame()
    prob['Tabela splitted'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'Tabela splitted': e}, ignore_index=True)
    
    return prob['Tabela splitted']
dados_palavras=tabela_palavras(nova)
nova.head()
"""
print('')

In [40]:
def cleanup(Column):
    
    Column= Column.replace(' " '," ").replace(" ' "," ").replace(' "'," ").replace('" '," ")\
    .replace('"'," ")
    
    Column= Column.replace("ç","c").replace("â","a").replace("ã","a").replace("à","a")\
    .replace("ê","e").replace("é","e").replace("ô","o").replace("õ","o").replace("/"," ")\
    .replace("?"," ").replace("!"," ").replace("@"," ").replace("\n"," ").replace(";"," ")\
    .replace(",","").replace(":"," ").replace("["," ").replace("]"," ").replace("}"," ")\
    .replace("{"," ").replace("\\"," ").replace(' " '," ").replace(" ' ","").replace(" * "," ")\
    .replace(" _ "," ").replace(" - "," ").replace(" . "," ").replace(" rt "," ").replace("…","")\
    .replace("“","").replace("             "," ").replace("   ","").replace(".."," ")\
    .replace("..."," ").replace("...."," ").replace("....."," ").replace("     "," ")\
    .replace("\ ","").replace("*","").replace(")","")  
    return Column 


In [41]:
dados.Treinamento = dados.Treinamento.apply(lambda x: x.lower())
#file_tre.Treinamento = file.Treinamento.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
dados.Treinamento = dados.Treinamento.apply(lambda x: x.translate(str.maketrans('','',string.digits)))
dados['Treinamento']=dados['Treinamento'].apply(cleanup)
 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
palavra=dados_palavras.value_counts(True) #probabilidade de sorteio das palavras que possuem a maior ocorrencia no dataframe
palavra.head()

bolsa    0.040958
e        0.033347
a        0.028504
de       0.027812
o        0.019095
Name: Tabela splitted, dtype: float64

In [29]:
nova = nova.B1.astype('category')

AttributeError: 'DataFrame' object has no attribute 'B1'

In [21]:
#Dataframe relevante 
relevantes = dados.loc[(dados['B1'])==1]
relevantes.set_index('B1',inplace=True)
relevantes.head()

,Treinamento
B1,
1,dolar cai a r$ 4 6 4 e bolsa avanca mais d...
1,albuquerque_af tradersclubbr entrei no final d...
1,bolsa cai dolar sobe o pais se desmancha 9 3...
1,brunosouzasc voce pode me explicar onde esta a...
1,📈 🇧 🇷 # ultimahora la bolsa de sao paulo ...


In [18]:
#Dataframe irrelevante
irrelevantes = dados.loc[(dados['B1'])==0]
irrelevantes.set_index('B1',inplace=True)
irrelevantes.head()

,Treinamento
B1,
0,gente eu amo esses perfuminhos da jequiti d le...
0,minha colega pediu um chiclete fui procurar na...
0,em 2 0 2 0 estamos com uma fila enorme no...
0,lulaoficial cuidar dos pobres e uma maravilha ...
0,uma das minhas maiores certezas na vida era qu...


In [22]:
#Split os tweets dos dataframes criados acima
relevante_separado=separador(relevantes,titulo)
irrelevante_separado=separador(irrelevantes,titulo)


In [28]:
b=pd.Series(irrelevante_separado.split())

AttributeError: 'DataFrame' object has no attribute 'split'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**